# Optimisation du modèle

Pour commencer, voici les performances actuelles de notre modèle, sans aucune optimisation. On prendra comme précision de base la moyenne de la précision sur 10 entraînements :

In [1]:
import numpy as np
from correction_brief import train_model

accuracies = []
for i in range(10):
    accuracies.append(train_model()["accuracy"])

baseline_accuracy = sum(accuracies) / len(accuracies)
print(f"Baseline accuracy: {baseline_accuracy}")

TRAINING MODEL...
MODEL TRAINED IN 0.7895596027374268 s
TRAINING MODEL...
MODEL TRAINED IN 0.8002817630767822 s
TRAINING MODEL...
MODEL TRAINED IN 0.7340793609619141 s
TRAINING MODEL...
MODEL TRAINED IN 0.7371559143066406 s
TRAINING MODEL...
MODEL TRAINED IN 0.767336368560791 s
TRAINING MODEL...
MODEL TRAINED IN 0.7426178455352783 s
TRAINING MODEL...
MODEL TRAINED IN 0.7250998020172119 s
TRAINING MODEL...
MODEL TRAINED IN 0.7265238761901855 s
TRAINING MODEL...
MODEL TRAINED IN 0.7336564064025879 s
TRAINING MODEL...
MODEL TRAINED IN 0.7453420162200928 s
Baseline accuracy: 0.7884130982367757


## Sélection des features

Afin de déterminer les features optimales à utiliser pour notre modèle, nous allons utiliser deux méthodes. Dans un premier temps, nous allons essayer des combinaisons de features aléatoires. Puis nous implémenterons un algorithme génétique (si nous avons le temps) pour réaliser la même tâche et comparerons les deux approches.

### Sélection aléatoire

Pour commencer, entraînons notre modèle 100 fois avec des features aléatoires et regardons quel est le meilleur résultat :

In [ ]:
import random

combination_counts = 2000

accuracies = []
filters = []

for f in range(combination_counts):
    features_filter = []
    for i in range(71): features_filter.append(bool(random.randint(0, 1)))
    acc = train_model(select_features=True, features_selector=features_filter, verbose=False)["accuracy"]
    accuracies.append(acc)
    filters.append(features_filter)
    if f%10 == 0: print(f"{f} features combinations tried")

best_accuracy = max(accuracies)
best_accuracy_index = accuracies.index(best_accuracy)
best_filter = filters[best_accuracy_index]

print(f"Best accuracy found: {best_accuracy}")
print(f"Parameters filter: {best_filter}")

In [2]:
best_accuracy_overall = 0.7909
best_features = [False, True, True, True, True, True, True, True, True, False, True, 
                 True, True, False, True, False, True, False, True, True, True, True, 
                 True, True, False, False, False, True, True, False, True, False, 
                 True, False, False, True, False, True, True, True, True, False, 
                 True, True, False, False, True, False, True, True, True, False, True,
                 True, False, True, True, True, False, False, False, True, True, False, 
                 True, False, True, True, False, False, True]

Conclusion : On observe une très légère amélioration de la précision avec les paramètres ci-dessus. On notera cependant que l'on a uniquement essayé une infime partie des combinaisons de paramètres possibles ici parmi les 2^71 combinaisons possibles !

## Optimisation des paramètres du modèle

Une étape importante et facile à mettre en place pour optimiser notre modèle est d'optimiser ses paramètres. Il y a plusieurs solutions pour cela (RandomSearchCV et GridSearchCV, notamment). Ici, nous allons générer des combinaisons de paramètres aléatoirement, comme un RandomSearchCV. On préférera ne pas prédéfinir de valeurs spécifiques pour les paramètres, comme un GridSearchCV. En effet, l'inconvénient de cette approche est qu'elle demande de définir une liste de valeurs potentielles pour chaque paramètre, et donc de se limiter dans la recherche des paramètres. Comme nous n'avons pas d'idées préalables concernant les valeurs adaptées pour nos paramètres, nous n'allons pas utiliser cette technique ici.


### Combinaisons de paramètres aléatoires

La structure de notre programme ne nous permettant pas d'utiliser RandomSearchCV simplement, nous allons nous-mêmes générer des combinaisons de paramètres aléatoires :

In [15]:
# Test de différentes combinaisons de paramètres pour un modèle SVC
import random


c_range = [-2, 1] # puissances de 10 à tester
kernel_range = ('linear', 'poly', 'rbf', 'sigmoid')
class_weight_range = (None, 'balanced')

combinations = []
results = []

for i in range(2000):
    c = random.uniform(1, 9) * 10**(random.randint(c_range[0], c_range[1]))
    kernel = random.choice(kernel_range)
    class_weight = random.choice(class_weight_range)
    params = {"C": c,
             "kernel": kernel,
             "class_weight": class_weight}
    print(params)
    combinations.append(params)
    result = train_model(select_features=True,
                              features_selector=best_features,
                              model_name="svm",
                              params=params)["accuracy"]
    print(f"Accuracy: {result}")
    results.append(result)


    
best_accuracy = max(results)
best_accuracy_index = results.index(best_accuracy)
best_params = combinations[best_accuracy_index]
print(f"Meilleure précision: {max(results)}")
print("Obtained with the following parameters:")
print(best_params)

if best_accuracy > best_accuracy_overall: 
    print("It is a new record!")
    best_accuracy_overall = best_accuracy
                                     
                


{'C': 0.02512990160882563, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10883069038391113 s
Accuracy: 0.5919395465994962
{'C': 78.40726016241648, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.12660884857177734 s
Accuracy: 0.8841309823677582
{'C': 0.02134749381828174, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.1339254379272461 s
Accuracy: 0.6372795969773299
{'C': 53.817441101087084, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 3.1204397678375244 s
Accuracy: 0.7984886649874056
{'C': 0.03215615243580604, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.0812983512878418 s
Accuracy: 0.7027707808564232
{'C': 72.70456718735765, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.12512493133544922 s
Accuracy: 0.8866498740554156
{'C': 7.33530185707136, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAI

MODEL TRAINED IN 0.3896775245666504 s
Accuracy: 0.7808564231738035
{'C': 3.226650891472465, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.11130785942077637 s
Accuracy: 0.7984886649874056
{'C': 10.74472045873077, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10266733169555664 s
Accuracy: 0.8513853904282116
{'C': 0.07624271968399954, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.19777870178222656 s
Accuracy: 0.5869017632241813
{'C': 0.8056023238085145, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.11532282829284668 s
Accuracy: 0.6977329974811083
{'C': 0.8452507427037951, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10277271270751953 s
Accuracy: 0.760705289672544
{'C': 1.1968953557127984, 'kernel': 'poly', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.1130516529083252 s
Accuracy: 0.7052896725440806
{'C': 

MODEL TRAINED IN 0.11315011978149414 s
Accuracy: 0.8312342569269522
{'C': 6.109788897300783, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.1093740463256836 s
Accuracy: 0.5390428211586902
{'C': 0.015915161083669912, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.2307746410369873 s
Accuracy: 0.5919395465994962
{'C': 0.40966589023348016, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.31435465812683105 s
Accuracy: 0.7405541561712846
{'C': 0.08645130849984227, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.6115379333496094 s
Accuracy: 0.6826196473551638
{'C': 50.59702754327671, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.14714646339416504 s
Accuracy: 0.8387909319899244
{'C': 0.43281108584432415, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.09306669235229492 s
Accuracy: 0.6700251889168766
{'C': 0

MODEL TRAINED IN 0.0923318862915039 s
Accuracy: 0.7506297229219143
{'C': 1.9914893286328397, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.2229902744293213 s
Accuracy: 0.7632241813602015
{'C': 0.3618487667455421, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.11463761329650879 s
Accuracy: 0.5390428211586902
{'C': 84.17330942247993, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.16717958450317383 s
Accuracy: 0.8841309823677582
{'C': 0.044462244519378684, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10776567459106445 s
Accuracy: 0.5944584382871536
{'C': 0.10015890660088074, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.11798858642578125 s
Accuracy: 0.6851385390428212
{'C': 0.04088869046610588, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.1710054874420166 s
Accuracy: 0.5944584382871536
{'C': 87

MODEL TRAINED IN 0.5503602027893066 s
Accuracy: 0.5994962216624685
{'C': 42.39674640269058, 'kernel': 'linear', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 4.333561658859253 s
Accuracy: 0.7959697732997482
{'C': 0.013927687074819674, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.20602822303771973 s
Accuracy: 0.5919395465994962
{'C': 0.766874164497218, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10835623741149902 s
Accuracy: 0.6926952141057935
{'C': 0.06055824095113283, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10868430137634277 s
Accuracy: 0.5969773299748111
{'C': 0.5385831730875099, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.10362863540649414 s
Accuracy: 0.6750629722921915
{'C': 0.8113909477409863, 'kernel': 'poly', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.09488487243652344 s
Accuracy: 0.6977329974811083

...

MODEL TRAINED IN 0.4918382167816162 s
Accuracy: 0.7909319899244333
{'C': 0.4882240058020824, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.10800576210021973 s
Accuracy: 0.5340050377833753
{'C': 57.82954696734941, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.1521298885345459 s
Accuracy: 0.8463476070528967
{'C': 6.729598316589964, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.09391927719116211 s
Accuracy: 0.8287153652392947
{'C': 3.4240186539987123, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.08517003059387207 s
Accuracy: 0.8035264483627204
{'C': 57.667797628111835, 'kernel': 'linear', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 3.8790223598480225 s
Accuracy: 0.7959697732997482
{'C': 1.1988906491736522, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.09023714065551758 s
Accuracy: 0.7707808564231738
{'C': 0.851

MODEL TRAINED IN 0.1817924976348877 s
Accuracy: 0.5793450881612091
{'C': 0.02029366983436999, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.0951988697052002 s
Accuracy: 0.5894206549118388
{'C': 50.02377493968595, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 2.3324599266052246 s
Accuracy: 0.7984886649874056
{'C': 5.6326118039171895, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.09486865997314453 s
Accuracy: 0.7506297229219143
{'C': 13.554495475723645, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.09956121444702148 s
Accuracy: 0.5390428211586902
{'C': 70.82056713039925, 'kernel': 'poly', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.17148041725158691 s
Accuracy: 0.853904282115869
{'C': 0.10788140193618397, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.08791851997375488 s
Accuracy: 0.7304785894206549
{'C': 0.847

MODEL TRAINED IN 0.1297905445098877 s
Accuracy: 0.8942065491183879
{'C': 0.06481160613300108, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.14805936813354492 s
Accuracy: 0.6675062972292192
{'C': 4.777033825264032, 'kernel': 'linear', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.48218679428100586 s
Accuracy: 0.7808564231738035
{'C': 0.24999006752251418, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.14658379554748535 s
Accuracy: 0.5440806045340051
{'C': 0.4406165892203743, 'kernel': 'poly', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.0989525318145752 s
Accuracy: 0.6700251889168766
{'C': 68.72476296329825, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 4.039197683334351 s
Accuracy: 0.801007556675063
{'C': 0.03162138006279177, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.2037971019744873 s
Accuracy: 0.576

MODEL TRAINED IN 0.46715354919433594 s
Accuracy: 0.7355163727959698
{'C': 8.887753467326847, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 1.1783933639526367 s
Accuracy: 0.7884130982367759
{'C': 0.023573372858155545, 'kernel': 'poly', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.0942690372467041 s
Accuracy: 0.5919395465994962
{'C': 0.0888572716093789, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.17446660995483398 s
Accuracy: 0.5617128463476071
{'C': 3.9853081187540385, 'kernel': 'linear', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.3977375030517578 s
Accuracy: 0.783375314861461
{'C': 0.2645669882091564, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.1598973274230957 s
Accuracy: 0.5415617128463476
{'C': 0.684993431889639, 'kernel': 'linear', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.16192626953125 s
Accuracy: 0.750629722921914

MODEL TRAINED IN 0.1549968719482422 s
Accuracy: 0.8513853904282116
{'C': 4.202561198502279, 'kernel': 'poly', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.09392666816711426 s
Accuracy: 0.7531486146095718
{'C': 71.7241608268528, 'kernel': 'rbf', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.12166547775268555 s
Accuracy: 0.8866498740554156
{'C': 3.665196455618144, 'kernel': 'sigmoid', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.0957341194152832 s
Accuracy: 0.5390428211586902
{'C': 0.1395759321749754, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.14398980140686035 s
Accuracy: 0.5541561712846348
{'C': 0.0404627628153328, 'kernel': 'rbf', 'class_weight': 'balanced'}
TRAINING MODEL...
MODEL TRAINED IN 0.12233948707580566 s
Accuracy: 0.6523929471032746
{'C': 0.011297915687655263, 'kernel': 'sigmoid', 'class_weight': None}
TRAINING MODEL...
MODEL TRAINED IN 0.1798863410949707 s
Accuracy: 0.6297229219143576


In [3]:
best_accuracy_overall = 0.8967
best_parameters = {'C': 38.238318053322125, 'kernel': 'rbf', 'class_weight': 'balanced'}

Modifier les hyperparamètres a significativement amélioré les performances de notre modèle, qui passe de 79% à 90% de précision.

## Comparaison avec d'autres modèles

Jusqu'ici, nous n'avons travaillé qu'avec des SVM. Voyons si d'autres modèles pourraient nous être utiles.

### KNeighborsClassifier

In [13]:
# Test du KNeighborsClassifier avec différents paramètres

import random


n_neighbors_range = [1, 15]
weights_range = ('uniform', 'distance')
algorithm_range = ('auto', 'ball_tree', 'kd_tree', 'brute')

combinations = []
results = []

for i in range(1000):
    n_neighbors = random.randint(n_neighbors_range[0], n_neighbors_range[1])
    weights = random.choice(weights_range)
    algorithm = random.choice(algorithm_range)
    params = {"n_neighbors": n_neighbors,
             "weights": weights,
             "algorithm": algorithm}
    print(params)
    combinations.append(params)
    result = train_model(select_features=True,
                              features_selector=best_features,
                              model_name="k_neighbors",
                              params=params)["accuracy"]
    print(f"Accuracy: {result}")
    results.append(result)


    
best_accuracy = max(results)
best_accuracy_index = results.index(best_accuracy)
best_params = combinations[best_accuracy_index]
print(f"Meilleure précision: {max(results)}")
print("Obtained with the following parameters:")
print(best_params)

if best_accuracy > best_accuracy_overall: 
    print("It is a new record!")
    best_accuracy_overall = best_accuracy


{'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0047283172607421875 s
Accuracy: 0.7808564231738035
{'n_neighbors': 2, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003686189651489258 s
Accuracy: 0.7884130982367759
{'n_neighbors': 2, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0035326480865478516 s
Accuracy: 0.7884130982367759
{'n_neighbors': 1, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007234811782836914 s
Accuracy: 0.7783375314861462
{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003091573715209961 s
Accuracy: 0.760705289672544
{'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007245779037475586 s
Accuracy: 0.7934508816120907
{'n_neighbors': 2, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN

Accuracy: 0.7808564231738035
{'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.004223346710205078 s
Accuracy: 0.7858942065491183
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005758523941040039 s
Accuracy: 0.801007556675063
{'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.004689693450927734 s
Accuracy: 0.7758186397984886
{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0033838748931884766 s
Accuracy: 0.760705289672544
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.0073626041412353516 s
Accuracy: 0.801007556675063
{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0029921531677246094 s
Accuracy: 0.760705289672544
{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'brute'}

MODEL TRAINED IN 0.006678104400634766 s
Accuracy: 0.7934508816120907
{'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005301952362060547 s
Accuracy: 0.783375314861461
{'n_neighbors': 13, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005173206329345703 s
Accuracy: 0.7858942065491183
{'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003135204315185547 s
Accuracy: 0.7934508816120907
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003526926040649414 s
Accuracy: 0.801007556675063
{'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006696939468383789 s
Accuracy: 0.7732997481108312
{'n_neighbors': 13, 'weights': 'distance', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.003152132034301758 s
Accuracy: 0.7909319899244333
{'n_neighbors': 6

Accuracy: 0.7884130982367759
{'n_neighbors': 1, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.004561901092529297 s
Accuracy: 0.7783375314861462
{'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.008784294128417969 s
Accuracy: 0.7909319899244333
{'n_neighbors': 8, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007484912872314453 s
Accuracy: 0.783375314861461
{'n_neighbors': 2, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.003017425537109375 s
Accuracy: 0.7884130982367759
{'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005642414093017578 s
Accuracy: 0.7758186397984886
{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005373954772949219 s
Accuracy: 0.7909319899244333
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'a

MODEL TRAINED IN 0.006336688995361328 s
Accuracy: 0.7707808564231738
{'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.002933025360107422 s
Accuracy: 0.7732997481108312
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005682945251464844 s
Accuracy: 0.801007556675063
{'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0027914047241210938 s
Accuracy: 0.783375314861461
{'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003757953643798828 s
Accuracy: 0.7934508816120907
{'n_neighbors': 1, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003309965133666992 s
Accuracy: 0.7783375314861462
{'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0034449100494384766 s
Accuracy: 0.7858942065491183
{'n_neighbors': 5, 'weight

...

MODEL TRAINED IN 0.004090547561645508 s
Accuracy: 0.7758186397984886
{'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007327079772949219 s
Accuracy: 0.783375314861461
{'n_neighbors': 8, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0031120777130126953 s
Accuracy: 0.7581863979848866
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007643222808837891 s
Accuracy: 0.7758186397984886
{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007444143295288086 s
Accuracy: 0.760705289672544
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0053253173828125 s
Accuracy: 0.7758186397984886
{'n_neighbors': 12, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0037717819213867188 s
Accuracy: 0.7858942065491183
{'n_neighbors': 

Accuracy: 0.7808564231738035
{'n_neighbors': 6, 'weights': 'uniform', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007023334503173828 s
Accuracy: 0.7758186397984886
{'n_neighbors': 11, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006806850433349609 s
Accuracy: 0.7808564231738035
{'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0029861927032470703 s
Accuracy: 0.7732997481108312
{'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007821798324584961 s
Accuracy: 0.7732997481108312
{'n_neighbors': 1, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0045015811920166016 s
Accuracy: 0.7783375314861462
{'n_neighbors': 7, 'weights': 'distance', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0035529136657714844 s
Accuracy: 0.7758186397984886
{'n_neighbors': 13, 'weights': 'uniform', 'algorithm': '

Accuracy: 0.7808564231738035
{'n_neighbors': 14, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005431652069091797 s
Accuracy: 0.7934508816120907
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006600141525268555 s
Accuracy: 0.7758186397984886
{'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0029320716857910156 s
Accuracy: 0.7732997481108312
{'n_neighbors': 13, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0034193992614746094 s
Accuracy: 0.7858942065491183
{'n_neighbors': 8, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.00716710090637207 s
Accuracy: 0.783375314861461
{'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006911039352416992 s
Accuracy: 0.7934508816120907
{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'br

Accuracy: 0.7884130982367759
{'n_neighbors': 7, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007252693176269531 s
Accuracy: 0.7707808564231738
{'n_neighbors': 11, 'weights': 'distance', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.004158735275268555 s
Accuracy: 0.7808564231738035
{'n_neighbors': 13, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.003835916519165039 s
Accuracy: 0.7858942065491183
{'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'ball_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.005847930908203125 s
Accuracy: 0.801007556675063
{'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006472110748291016 s
Accuracy: 0.7808564231738035
{'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'auto'}
TRAINING MODEL...
MODEL TRAINED IN 0.0030770301818847656 s
Accuracy: 0.7758186397984886
{'n_neighbors': 13, 'weights': 'uniform', 'algorithm': 'kd_t

Accuracy: 0.7783375314861462
{'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0028557777404785156 s
Accuracy: 0.7884130982367759
{'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'brute'}
TRAINING MODEL...
MODEL TRAINED IN 0.0038471221923828125 s
Accuracy: 0.7732997481108312
{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.0071446895599365234 s
Accuracy: 0.760705289672544
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.007535457611083984 s
Accuracy: 0.7758186397984886
{'n_neighbors': 2, 'weights': 'uniform', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006889820098876953 s
Accuracy: 0.7884130982367759
{'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'kd_tree'}
TRAINING MODEL...
MODEL TRAINED IN 0.006801128387451172 s
Accuracy: 0.7758186397984886
{'n_neighbors': 11, 'weights': 'uniform', 'algorithm': '

Ce modèle nous permet d'atteindre, au mieux, une précision de 80%. Il n'est donc pas intéressant pour nous.

### SGDClassifier

In [14]:
# Test du SGDClassifier avec différents paramètres

import random


loss_range = ('hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge','perceptron')
penalty_range = ('l2', 'l1', 'elasticnet')
alpha_range = [-5, 1] # Puissance de 10 pour le paramètre alpha

combinations = []
results = []

for i in range(1000):
    alpha = random.uniform(1, 9) * 10**(random.randint(alpha_range[0], alpha_range[1]))
    loss = random.choice(loss_range)
    penalty = random.choice(penalty_range)
    params = {"alpha": alpha,
             "penalty": penalty,
             "loss": loss}
    print(params)
    combinations.append(params)
    result = train_model(select_features=True,
                              features_selector=best_features,
                              model_name="sgd",
                              params=params)["accuracy"]
    print(f"Accuracy: {result}")
    results.append(result)


    
best_accuracy = max(results)
best_accuracy_index = results.index(best_accuracy)
best_params = combinations[best_accuracy_index]
print(f"Meilleure précision: {max(results)}")
print("Obtained with the following parameters:")
print(best_params)

if best_accuracy > best_accuracy_overall: 
    print("It is a new record!")
    best_accuracy_overall = best_accuracy

{'alpha': 3.6594715731731506, 'penalty': 'l1', 'loss': 'log_loss'}
TRAINING MODEL...
MODEL TRAINED IN 0.01814103126525879 s
Accuracy: 0.5088161209068011
{'alpha': 2.972348417435365e-05, 'penalty': 'elasticnet', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.08676719665527344 s
Accuracy: 0.7304785894206549
{'alpha': 0.0008056906350954482, 'penalty': 'l2', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.029924392700195312 s
Accuracy: 0.7304785894206549
{'alpha': 7.140575833234458, 'penalty': 'elasticnet', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.01860332489013672 s
Accuracy: 0.5088161209068011
{'alpha': 0.16319329354658088, 'penalty': 'l2', 'loss': 'log_loss'}
TRAINING MODEL...
MODEL TRAINED IN 0.015386819839477539 s
Accuracy: 0.6624685138539043
{'alpha': 8.932157657913443e-05, 'penalty': 'elasticnet', 'loss': 'hinge'}
TRAINING MODEL...


/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


MODEL TRAINED IN 0.0811777114868164 s
Accuracy: 0.7304785894206549
{'alpha': 0.0390962524465484, 'penalty': 'l2', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.019368648529052734 s
Accuracy: 0.6952141057934509
{'alpha': 0.07867200022785997, 'penalty': 'l2', 'loss': 'squared_hinge'}
TRAINING MODEL...


/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


MODEL TRAINED IN 0.22406315803527832 s
Accuracy: 0.6498740554156172
{'alpha': 6.940061122947295, 'penalty': 'l2', 'loss': 'log_loss'}
TRAINING MODEL...
MODEL TRAINED IN 0.013285160064697266 s
Accuracy: 0.5692695214105793
{'alpha': 5.709403721563692, 'penalty': 'elasticnet', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.01768946647644043 s
Accuracy: 0.491183879093199
{'alpha': 82.57990564052992, 'penalty': 'l1', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.031982421875 s
Accuracy: 0.5088161209068011
{'alpha': 8.27535393136398e-05, 'penalty': 'elasticnet', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.08084535598754883 s
Accuracy: 0.6171284634760705
{'alpha': 0.5888250030785657, 'penalty': 'l2', 'loss': 'perceptron'}
TRAINING MODEL...
MODEL TRAINED IN 0.01174163818359375 s
Accuracy: 0.6448362720403022
{'alpha': 0.40295703405932265, 'penalty': 'l1', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.018514156341552734 s
Accuracy: 0.491183879093199

...

/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


MODEL TRAINED IN 0.21933889389038086 s
Accuracy: 0.7481108312342569
{'alpha': 0.28021523075652927, 'penalty': 'l2', 'loss': 'log_loss'}
TRAINING MODEL...
MODEL TRAINED IN 0.013724565505981445 s
Accuracy: 0.6624685138539043
{'alpha': 5.452033135688238e-05, 'penalty': 'elasticnet', 'loss': 'perceptron'}
TRAINING MODEL...
MODEL TRAINED IN 0.08011722564697266 s
Accuracy: 0.6624685138539043
{'alpha': 3.664671555571955, 'penalty': 'elasticnet', 'loss': 'squared_hinge'}
TRAINING MODEL...
MODEL TRAINED IN 0.019474029541015625 s
Accuracy: 0.5088161209068011
{'alpha': 0.006012833736569476, 'penalty': 'elasticnet', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.04441666603088379 s
Accuracy: 0.7052896725440806
{'alpha': 0.0008351708747086481, 'penalty': 'elasticnet', 'loss': 'log'}
TRAINING MODEL...
MODEL TRAINED IN 0.055023908615112305 s
Accuracy: 0.6675062972292192
{'alpha': 0.063913521048036, 'penalty': 'l2', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.023787498474121094

/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


MODEL TRAINED IN 0.030529499053955078 s
Accuracy: 0.6120906801007556
{'alpha': 0.0741498766094346, 'penalty': 'elasticnet', 'loss': 'perceptron'}
TRAINING MODEL...
MODEL TRAINED IN 0.01724553108215332 s
Accuracy: 0.5717884130982368
{'alpha': 0.8296990238075245, 'penalty': 'elasticnet', 'loss': 'squared_hinge'}
TRAINING MODEL...
MODEL TRAINED IN 0.3470473289489746 s
Accuracy: 0.5919395465994962
{'alpha': 0.00042804744636489104, 'penalty': 'l1', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.06226396560668945 s
Accuracy: 0.7229219143576826
{'alpha': 0.844550382427179, 'penalty': 'l1', 'loss': 'hinge'}
TRAINING MODEL...
MODEL TRAINED IN 0.018114328384399414 s
Accuracy: 0.491183879093199
{'alpha': 3.111940832172312e-05, 'penalty': 'l1', 'loss': 'squared_hinge'}
TRAINING MODEL...
MODEL TRAINED IN 0.08783745765686035 s
Accuracy: 0.72544080604534
{'alpha': 37.743286246116206, 'penalty': 'l1', 'loss': 'modified_huber'}
TRAINING MODEL...
MODEL TRAINED IN 0.047086238861083984 s
A

/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vincent/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


On obtient 79,8% de précision avec ce modèle, dans le meilleur des cas. Il n'est donc pas intéressant par rapport à un SVM.

## Un petit état des lieux

Voici actuellement, les valeurs optimales que nous avons définies pour notre modèle :

In [5]:
best_features = [False, True, True, True, True, True, True, True, True, False, True, 
                 True, True, False, True, False, True, False, True, True, True, True, 
                 True, True, False, False, False, True, True, False, True, False, 
                 True, False, False, True, False, True, True, True, True, False, 
                 True, True, False, False, True, False, True, True, True, False, True,
                 True, False, True, True, True, False, False, False, True, True, False, 
                 True, False, True, True, False, False, True]
best_model = "svm"
best_parameters = {'C': 38.238318053322125, 'kernel': 'rbf', 'class_weight': 'balanced'}
best_accuracy_overall = 0.8967

Afin d'aller plus loin dans l'optimisation de notre modèle, nous allons maintenant nous pencher sur le concept de programmation génétique.

**Définition** : "La programmation génétique est une méthode automatique inspirée par le mécanisme de la sélection naturelle tel qu'il a été établi par Charles Darwin pour expliquer l'adaptation plus ou moins optimale des organismes à leur milieu. Elle a pour but de trouver par approximations successives des programmes répondant au mieux à une tâche donnée." *(source : Wikipédia)*

Dans un premier temps, nous allons appliquer cette méthode à la sélection des features du modèle.

## Sélection des features à l'aide de la programmation génétique

On considèrera qu'une population est constituée de combinaisons possibles de paramètres qui se présentent sous la forme de listes de booléens (on rappelle qu'une feature est utilisée si sa valeur correspondante dans le vecteur vaut True, et reste inutilisée sinon).

In [ ]:
import random
import pandas as pd

population_size = 120 # Nombre de vecteurs de paramètres dans la population initiale
stop_if_record_is_reached = False # Définit si l'on doit s'arrêter ou non si l'on atteint
# notre record de précision
n_iterations = 30 # Nombre d'itérations maximal à exécuter

### INUTILISÉ POUR L'INSTANT ###
mutation_chance = 0.25 # Probabilité qu'un vecteur de la population mute. Une feature aléatoire
# de ce vecteur change alors
################################


def cross_features(features_array):
    """Cette fonction croise deux listes de specimens (ici des vecteurs de features)
    pour en créer de nouveaux"""
    
    children = {"features": []}
    
    parents_1 = features_array.iloc[:len(features_array)//2]
    parents_2 = features_array.iloc[len(features_array)//2:]
    
    for row1, row2 in zip(parents_1.values, parents_2.values):
        child = []
        for index in range(len(row1[0])):
            if random.randint(0, 1) == 1: child.append(row1[0][index])
            else: child.append(row2[0][index])
        children["features"].append(child)
    return pd.DataFrame(children)
    
    

def optimize_features():
    
    """Optimise la sélection des features à l'aide de techniques de programmation génétique"""
    
    current_population = {"features": [],
                         "accuracy": []}
    current_best = ()

    # Création d'une population initiale et test de sa précision

    for i in range(population_size):
        features_filter = []
        for i in range(71): features_filter.append(bool(random.randint(0, 1)))
        acc = train_model(select_features=True, 
                          features_selector=features_filter, 
                          model_name=best_model,
                          params=best_parameters,
                          verbose=False)["accuracy"]
        
            
        current_population["features"].append(features_filter)
        current_population["accuracy"].append(acc)
  
    best_acc = max(current_population["accuracy"])
    acc_index = current_population["accuracy"].index(best_acc)
    best_vector = current_population["features"][acc_index]
    current_best = (best_vector, best_acc)
    print("Current best features:")
    print(current_best[1])
    
    if stop_if_record_is_reached and best_acc > best_accuracy_overall:
        return current_best
    
    # Commencer les itérations d'évolution
    
    for i in range(n_iterations): 
        
        # On garde la moitié des vecteurs de features la plus performante
        population = pd.DataFrame(current_population)
        population.sort_values(by="accuracy", ascending=False, inplace=True)
        best_half = population.iloc[:population_size//2]
        best_half.drop(columns=["accuracy"], inplace=True)
        best_half.reset_index(drop=True, inplace=True)
        
        # On croise les 50% les plus performants pour créer de nouveaux vecteurs (25% du nouveau total)
        children_features = cross_features(best_half)
        
        # On ajoute 25% de nouveaux vecteurs aléatoires
        random_features = {"features": []}
        for i in range(population_size//4):
            features_filter = []
            for i in range(71): features_filter.append(bool(random.randint(0, 1)))
            random_features["features"].append(features_filter)
            
        random_features = pd.DataFrame(random_features)
        
        current_population = pd.concat([best_half, children_features, random_features], ignore_index=True)
        
        # On calcule à nouveau les performances
        
        accuracies = []
        for row in current_population.values:
            acc = train_model(select_features=True, 
                          features_selector=row[0], 
                          model_name=best_model,
                          params=best_parameters,
                          verbose=False)["accuracy"]
            accuracies.append(acc)
        current_population["accuracy"] = accuracies
        

        best_acc = max(current_population["accuracy"])
        best_vector = current_population[current_population["accuracy"] == best_acc].iloc[0]["features"]
        current_best = (best_vector, best_acc)
        print("Current best accuracy:")
        print(current_best[1])
        if stop_if_record_is_reached and best_acc > best_accuracy_overall:
            return current_best
        
    return current_best

        
result = optimize_features()
print("RESULTS OF FEATURES SELECTION WITH GENETIC ALGORITHM: ")
print(f"Accuracy: {result[1]}")
print(f"Corresponding features: {result[0]}")

if result[1] > best_accuracy_overall:
    print("It is a new record!")
    best_accuracy_overall = best_accuracy

Current best features:
0.8866498740554156


/tmp/ipykernel_21968/1807570695.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_half.drop(columns=["accuracy"], inplace=True)


Avec cette technique, on a amélioré notre record de précision de 1,5% environ.

In [ ]:
# Paramètres optimaux :
best_features = [False, True, True, True, True, True, True, True, True, False, True, 
                 True, True, False, True, False, True, False, True, True, True, True, 
                 True, True, False, False, False, True, True, False, True, False, 
                 True, False, False, True, False, True, True, True, True, False, 
                 True, True, False, False, True, False, True, True, True, False, True,
                 True, False, True, True, True, False, False, False, True, True, False, 
                 True, False, True, True, False, False, True]
best_model = "svm"
best_parameters = {'C': 38.238318053322125, 'kernel': 'rbf', 'class_weight': 'balanced'}
best_accuracy_overall = 0.8967